In [2]:
# Initial Imports 
import praw 
import os 
import pandas as pd 
from datetime import datetime

# Stablish Credential to use Reddit API
reddit = praw.Reddit(
    client_id = os.getenv('REDDIT_CLIENT_ID'),
    client_secret = os.getenv('REDDIT_CLIENT_SECRET'),
    password = os.getenv('REDDIT_PASSWORD'),
    username = os.getenv('REDDIT_USERNAME'),
    user_agent ='testscript by /u/fakebot3'
)
print(reddit.user.me())

jaimebunay


In [78]:
# Define function to retrieve subreddits.
"""
BTC, XRP, XLM,LTC,EOS,ETH,BCH,BSV,XMR, TRX. 
['Bitcoin', 'BitcoinCash', 'BitcoinSV' 'Ethereum', 'Monero', 'Litecoin', 'EOS','Tron']
FINAL LIST: BTC, XRP, XLM,LTC,EOS,ETH,BCH,BSV,XMR, TRX.
"""
def get_subreddits(): 
    # List of top ten traded cryptocurrecies
    crypto_list = ['bitcoin+btc', 'ethereum+eth', 'xrp', 'tether+usdt','bitcoincash+bch', 
                   'bitcoinsv+bsv', 'litecoin+ltc', 'eos','monero+xmr', 'tron+trx']
    # Bitcoin SV
    # List that will hold all subreddit objects
    subreddits = []
    
    # Iterate through crypto list
    # retrieve new subreddit
    for crypto in crypto_list: 
        submissions = reddit.subreddit(crypto).new(limit=10000)
        # local list to hold subreddit object
        posts = []
        
        # Iterate through the submissions object and append to posts
        for sub in submissions: 
            posts.append(sub)
            
        # Append list of posts to subreddits
        subreddits.append(posts)
        
    return subreddits 

In [79]:
def calculate_votes(ratio, score): 
    ups = round((ratio * score)/(2*ratio - 1)) if ratio != 0.5 else round(score/2)
    downs = ups - score   
    return ups, downs
    

In [64]:
subreddit_list = get_subreddits()

In [80]:
subreddit_list[1][3].subreddit.title[:15]

'Ethereum'

In [72]:
# Itererate through subreddit lists to find posts relating to each coin
crypto_subreddits = pd.DataFrame()
coin_list=['Bitcoin','Ethereum','XRP','Tether','BitcoinCash','BitcoinSV','Litecoin','EOS','Monero','Tron']
index = 0
for posts in subreddit_list: 
    # Iterate through each posts to pull features
    dates = []
    text = []
    up_votes = []
    down_votes = []
    num_comments = []
    for post in posts:
        if post.selftext =='': 
            pass 
        else: 
            dates.append(datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'))
            text.append(post.selftext)
            
            # Get the number of up_votes and down_votes
            # append to list
            up_vote, down_vote = calculate_votes(post.upvote_ratio, post.score)
            up_votes.append(up_vote)
            down_votes.append(down_vote)
            num_comments.append(post.num_comments)
    # Add features to dataframe
    
    crypto_subreddits[coin_list[index], 'Date'] = pd.Series(dates)
    crypto_subreddits[coin_list[index],'Text'] = pd.Series(text)
    crypto_subreddits[coin_list[index],'up_votes'] = pd.Series(up_votes)
    crypto_subreddits[coin_list[index],'down_votes'] = pd.Series(down_votes)
    crypto_subreddits[coin_list[index],'num_comments'] = pd.Series(num_comments)
    
    index += 1      


In [73]:
crypto_subreddits

,"(Bitcoin, Date)","(Bitcoin, Text)","(Bitcoin, up_votes)","(Bitcoin, down_votes)","(Bitcoin, num_comments)","(Ethereum, Date)","(Ethereum, Text)","(Ethereum, up_votes)","(Ethereum, down_votes)","(Ethereum, num_comments)",...,"(Monero, Date)","(Monero, Text)","(Monero, up_votes)","(Monero, down_votes)","(Monero, num_comments)","(Tron, Date)","(Tron, Text)","(Tron, up_votes)","(Tron, down_votes)","(Tron, num_comments)"
0,2020-04-13,I have coins I want to move off Coinbase. Any...,2,0,3,2020-04-13,Besides Mycrypto and MEW...,6.0,1.0,4.0,...,2020-04-13,I just finished downloading the blockchain and...,3.0,0.0,2.0,2020-04-13,I've been looking for two hours and i've gotte...,1.0,0.0,6.0
1,2020-04-13,Hello ledger live on my pc always show the wro...,1,0,0,2020-04-13,"Hi guys, \n\nthis is a question for all of you...",4.0,1.0,0.0,...,2020-04-13,[deleted],3.0,0.0,4.0,2020-04-13,Batten down the toilet roll because this one i...,12.0,1.0,0.0
2,2020-04-13,So that really does help,7,0,4,2020-04-12,"Hi, there's something fundamental i don't quit...",5.0,0.0,2.0,...,2020-04-13,"As I understand it, Syncing up all the Daemon ...",2.0,1.0,7.0,2020-04-13,I'm currently playing Evolution on PlayStatio...,1.0,0.0,0.0
3,2020-04-13,How can I buy BTC or ETH online with card? See...,2,0,2,2020-04-12,CryptoPersonalities are pixel characters that ...,15.0,5.0,1.0,...,2020-04-13,"Hi guys this is a guide in Spanish about ""how ...",44.0,3.0,0.0,2020-04-12,so i really wanna join the tron fandom because...,11.0,0.0,2.0
4,2020-04-13,I received an email from a guy ekkjoaquinzce a...,0,0,6,2020-04-11,Could Bitcoin Cash wallet functionality be bui...,5.0,4.0,5.0,...,2020-04-13,Given the success of the previous MAAMs ([see ...,20.0,0.0,21.0,2020-04-12,Give these 3 TRX exercises a whirl🌪. They targ...,14.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2020-03-26,"Hi, I just wanted to let everyone know of some...",5,2,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,2020-03-26,"Hey all,\n\nI sent a very small amount of BTC ...",21,6,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,2020-03-26,Maybe Flipstarter needs a web interface. \n\nC...,14,11,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681,2020-03-26,"Hello dear BTC followers, \n\nWe have been try...",0,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
crypto_subreddits.columns = pd.MultiIndex.from_tuples(crypto_subreddits.columns)

In [82]:
crypto_subreddits.to_csv('current_data.csv')

In [236]:
crypto_subreddits.head()

Bitcoin                                                              \
         Date                                               Text up_votes   
0  2020-04-13  [coin, want, move, coinbase, recommendation, t...        2   
1  2020-04-13  [hello, ledger, live, pc, always, show, wrong,...        1   
2  2020-04-13                                [really, doe, help]        7   
3  2020-04-13  [buy, btc, eth, online, card, seems, like, eve...        2   
4  2020-04-13  [received, email, guy, ekkjoaquinzce, say, lea...        0   

                             Ethereum  \
  down_votes num_comments        Date   
0          0            3  2020-04-13   
1          0            0  2020-04-13   
2          0            4  2020-04-12   
3          0            2  2020-04-12   
4          0            6  2020-04-11   

                                                                          \
                                                Text up_votes down_votes   
0                        Besides Mycrypto and MEW...      6.0        1.0   
1  Hi guys, \n\nthis is a question for all of you...      4.0        1.0   
2  Hi, there's something fundamental i don't quit...      5.0        0.0   
3  CryptoPersonalities are pixel characters that ...     15.0        5.0   
4  Could Bitcoin Cash wallet functionality be bui...      5.0        4.0   

                ...      Monero  \
  num_comments  ...        Date   
0          4.0  ...  2020-04-13   
1          0.0  ...  2020-04-13   
2          2.0  ...  2020-04-13   
3          1.0  ...  2020-04-13   
4          5.0  ...  2020-04-13   

                                                                          \
                                                Text up_votes down_votes   
0  I just finished downloading the blockchain and...      3.0        0.0   
1                                          [deleted]      3.0        0.0   
2  As I understand it, Syncing up all the Daemon ...      2.0        1.0   
3  Hi guys this is a guide in Spanish about "how ...     44.0        3.0   
4  Given the success of the previous MAAMs ([see ...     20.0        0.0   

                      Tron                                                     \
  num_comments        Date                                               Text   
0          2.0  2020-04-13  I've been looking for two hours and i've gotte...   
1          4.0  2020-04-13  Batten down the toilet roll because this one i...   
2          7.0  2020-04-13  I'm currently playing  Evolution on PlayStatio...   
3          0.0  2020-04-12  so i really wanna join the tron fandom because...   
4         21.0  2020-04-12  Give these 3 TRX exercises a whirl🌪. They targ...   

                                    
  up_votes down_votes num_comments  
0      1.0        0.0          6.0  
1     12.0        1.0          0.0  
2      1.0        0.0          0.0  
3     11.0        0.0          2.0  
4     14.0        2.0          5.0  

[5 rows x 50 columns]

In [ ]:
for value in crypto_subreddits[('Bitcoin','Text')]: 
    print(value)

In [228]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# Code to download wordnet corpora
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaime\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [261]:
def process_text(doc):
    doc = str(doc)
    sw = set(stopwords.words('english'))
    regex = re.compile('[^a-zA-Z ]')
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    #lemm_words = [lemmatizer.lemmatize(word) for word in words]
    final_words = [word.lower() for word in words if word.lower() not in sw]
   # returns a large string of all words from all articles in corpus
    return final_words

In [258]:
a_list = 'my name is jaime, my name is mike &*)'
clean = process_text(a_list)
clean

['name', 'jaime', 'name', 'mike']

In [ ]:
crypto_subreddits.dtypes

In [262]:
for row, column in crypto_subreddits.items():
    if column.name[1] == 'Text':
        #print(column.name)
        #print(row)
        #print(column.name)
        #print(row, column)
        # print(column.values)
        crypto_subreddits[column.name] = crypto_subreddits[column.name].apply(lambda x: process_text(x))

In [265]:
crypto_subreddits.to_csv('current_data.csv')